# A notebook to move samples into a location

In [1]:
### a two columns file with col1: sampleID, col2: storage location
CSVfilename="/Users/pierrespc/Documents/PostDocPasteur/aDNA/Import_eLAB/API_FUNCTIONALITIES/UploadData/assignStorage/Tables/2ndBatchPetrousToTartu.txt"
delimiterFile="\t"

## Preparing the note book

Please enter the one-line file where your token is saved in the following cell

In [2]:
tokenFile="/Users/pierrespc/Documents/PostDocPasteur/aDNA/Import_eLAB/API_FUNCTIONALITIES/credentials/tokenELAB"
import os
import json
import requests
import csv
import pandas
import numpy
from apiclient import discovery, errors
from httplib2 import Http
from oauth2client import client, file, tools
import os.path

token = format(open(tokenFile,"r").readline().strip())
url = "https://elab-dev.pasteur.fr/api/v1/"
headers1 = {'Authorization': token, 'Accept': 'application/json','Content-Type':'application/json'}
headers2 = {'Authorization': token, 'Accept': 'application/json'}



def BadRequest(myReq,code=200):
    return(myReq.status_code !=code)



In [3]:
storageByID={}
r=requests.get(url+"/storageLayers",headers=headers1)
stoData=r.json().get("data")
for sto in stoData:
    storageByID[sto["storageLayerID"]]={"name":sto["name"],"parentID":sto["parentStorageLayerID"]}
    #print(sto["name"])
def getParentSto(ID,stoDict):
    if stoDict[ID]["parentID"]==0:
        return(stoDict[ID]["name"])
    else:
        return(getParentSto(stoDict[ID]["parentID"],stoDict)+", "+stoDict[ID]["name"])
    
    
storage={}
for stoID in storageByID.keys():
    name=getParentSto(stoID,storageByID)
    storage[name]=stoID
    

print(storage)

{'Nico office': 774657, 'In Copenhagen': 774658, 'Tom Gilbert Freezer': 774659, 'Tom Gilbert Freezer, bag A1 + A2': 774671, 'Tom Gilbert Freezer, bag A1 + A2, calculus extraction': 774677, 'Tom Gilbert Freezer, bag A1 + A2, petrous back-up': 774678, 'Tom Gilbert Freezer, bag A1 + A2, petrous extraction': 774679, 'Tom Gilbert Freezer, bag A1 + A2, pulp back-up': 774680, 'Tom Gilbert Freezer, bag A1 + A2, pulp extraction': 774681, 'Tom Gilbert Freezer, bag A1 + A2, root back-up': 774682, 'Tom Gilbert Freezer, bag A1 + A2, root extraction': 774683, 'Tom Gilbert Freezer, bag A1 + A2, C group sensitive, blue box, back-up': 774684, 'Tom Gilbert Freezer, bag Mariano Del Papa calculus': 775907, 'Tom Gilbert Freezer, bag Mariano Del Papa calculus, Mariano Del Papa calculus extraction': 775908, 'Tom Gilbert Freezer, bag A1 + A2, already processed': 775909, 'Freezer n9': 774661, 'Freezer n9, Miren Drawer 2': 774674, 'Freezer n9, Miren Drawer 2, Blue Rack 1': 774675, 'Freezer 4': 774836, 'Freezer 

Prepare all the eLab-API keys necessary to down and upload data

In [4]:
def BadRequest(myReq,code=200):
    return(myReq.status_code !=code)


r = requests.get(url + "sampleTypes", headers = headers2)
if BadRequest(r,200):
    print("Bad request")
data = r.json()
types = {}
for typ in data.get("data"):
    types[format(typ.get("name"))] = format(typ.get("sampleTypeID"))

print(types)

r = requests.get(url + "sampleTypes/" + types["Extract"] + "/meta", headers = headers2)
if BadRequest(r,200):
    print("Bad request")
data = r.json()
FeateLabExe = {}
for feat in ['Name','Description','Note','Amount','Unit',"parentSampleID"]:
    FeateLabExe[feat] = {"ID": "notMeta"}
for feat in data.get("data"):
    FeateLabExe[format(feat.get("key"))] = { "ID":format(feat.get("sampleTypeMetaID")),
                                              "TYPE":format(feat.get("sampleDataType"))}

{'Individual': '39466', 'Site': '39468', 'Skeleton Element': '39469', 'Extract': '39470', 'Indexed Library': '39494', 'Library pool': '39495', 'Non Indexed Library': '39556', 'Bone pellet': '39599'}


Now we get all registered ID for all types.
(I am lazy now to try to find a clever way to process only the types we need downstream)

In [5]:
registered = {}
for name in types:
    ID=types[name]
    print(name + " --> " + format(ID))
    r = requests.get(url + "samples" , headers = headers2, params = {'sampleTypeID': ID})
    if BadRequest:
        r.raise_for_status()
    data = r.json()
    myList = {}
    for sam in data.get("data"):
        if format(sam.get("name")) in myList.keys():
            print(name + ": " + sam.get("name") + " duplicated")
            break
        myList[format(sam.get("name"))]=format(sam.get("sampleID"))
    registered[name] = myList
print("finished")



Individual --> 39466
Site --> 39468
Skeleton Element --> 39469
Extract --> 39470
Indexed Library --> 39494
Library pool --> 39495
Non Indexed Library --> 39556
Bone pellet --> 39599
finished


Read the file with storage specifications and iterize to move sample

In [6]:
Table=pandas.read_csv(CSVfilename,delimiter=delimiterFile)
Table

,ID,Location
0,AR0681.2,In Tartu
1,AR0696.1,In Tartu
2,AR0309.2,In Tartu
3,AR0284.2,In Tartu
4,AR0301.2,In Tartu
5,AR0316.2,In Tartu
6,AR0313.2,In Tartu
7,AR0288.1,In Tartu
8,AR0449.2,In Tartu
9,AR0462.1,In Tartu


In [7]:
import re
patternDict={"Skeleton Element":'[A][R][0-9][0-9][0-9][0-9][.][0-9]',
         "Extract":'[A][R][0-9][0-9][0-9][0-9][.][0-9][.][0-9]'}


for index,name in Table["ID"].items():
    Loc=Table.loc[index,"Location"]
    sampType="??"
    for typKey,pattern in patternDict.items():
        if re.match(pattern,name):
            sampType=typKey
    if sampType == "??":
        print(name+" has not a recognized pattern and is skipped")
    #print(sampType)
    if name not in registered[sampType]:
        print(name+" ("+typKey+") is not found in eLab and is skipped")
    if Loc not in storage.keys():
            print(name+ " has wriong location ("+Loc+") and is skipped")
    idSam=format(registered[sampType][name])
    idLoc=format(storage[Loc])
    r = requests.post(url + "samples/"+idSam+"/moveToLayer/"+idLoc, headers = headers2, params = {})
    if BadRequest(r,204):
        print(name+" "+Loc)
        r.raise_for_status()
        
print("finished")

finished
